In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
# 定义起始时期
def get_custom_week_period(date):
    start_date = pd.Timestamp('2025-01-02')
    delta_days = (date - start_date).days
    week_number = delta_days // 7
    period_start = start_date + pd.Timedelta(days=week_number * 7)
    period_end = period_start + pd.Timedelta(days=6)
    return f"{period_start.strftime('%Y-%m-%d')}至{period_end.strftime('%Y-%m-%d')}"

In [3]:
def process_excel(input_path, output_folder):
    # 读取日报数据
    df = pd.read_excel(input_path, sheet_name='日报by渠道')
    df2 = pd.read_excel(input_path, sheet_name='日报by优惠券')    

    df['时期'] = df['日期'].apply(get_custom_week_period)

    # 按周周期进行groupby汇总
    weekly_report = df.groupby(['时期','城市']).agg({
        '触达用户数': 'sum',
        '点击用户数': 'sum',
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '交易订单量': 'sum',
        '核销用户数': 'sum',
        '交易新用户数': 'sum',
        '券补金额': 'sum',
        '渠道成本': 'sum'
    }).reset_index()

    weekly_report['未核销用户数'] = weekly_report['交易用户数'] - weekly_report['核销用户数']
    weekly_report['触达点击率'] = weekly_report['点击用户数'] / weekly_report['触达用户数']
    weekly_report['点击转化率'] = weekly_report['交易用户数'] / weekly_report['点击用户数']
    weekly_report['支付核销率'] = weekly_report['核销用户数'] / weekly_report['交易用户数']
    weekly_report['日均交易订单量'] = weekly_report['交易订单量'] / 7
    weekly_report['团单CAC'] = (weekly_report['券补金额'] + weekly_report['渠道成本']) / weekly_report['交易用户数']

    rate_columns = ['触达点击率', '点击转化率', '支付核销率']
    weekly_report[rate_columns] = weekly_report[rate_columns].apply(lambda x: (x * 100).round(2).astype(str) + '%')

    decimal_columns = ['日均交易订单量', '团单CAC']
    weekly_report[decimal_columns] = weekly_report[decimal_columns].round(2)

    wb = Workbook()

    ws1 = wb.active
    ws1.title = '线上获客概览'

    for r in dataframe_to_rows(weekly_report, index=False, header=True):
        ws1.append(r)

    cac_report = df.groupby(['时期', '城市','渠道']).agg({
        '渠道成本': 'sum',
        '券补金额': 'sum',
        '交易用户数': 'sum',
        '核销用户数': 'sum'
    }).reset_index()
    cac_report['总花费'] = cac_report['渠道成本'] + cac_report['券补金额']
    cac_report['团单CAC'] = cac_report['总花费'] / cac_report['交易用户数']
    cac_report['核销CAC'] = cac_report['总花费'] / cac_report['核销用户数']

    decimal_columns2 = ['总花费', '团单CAC', '核销CAC']
    cac_report[decimal_columns2] = cac_report[decimal_columns2].round(2)

    ws2 = wb.create_sheet(title='CAC')
    for r in dataframe_to_rows(cac_report, index=False, header=True):
        ws2.append(r)

    conversion_report = df.groupby(['时期','城市', '渠道']).agg({
        '触达用户数': 'sum',
        '点击用户数': 'sum',
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '交易订单量': 'sum',
        '核销用户数': 'sum',
        '交易新用户数': 'sum',
        '券补金额': 'sum',
        '渠道成本': 'sum'
    }).reset_index()

    conversion_report['触达点击率'] = conversion_report['点击用户数'] / conversion_report['触达用户数']
    conversion_report['点击转化率'] = conversion_report['交易用户数'] / conversion_report['点击用户数']
    conversion_report['新用户占比'] = conversion_report['交易新用户数'] / conversion_report['交易用户数']

    rate_columns2 = ['触达点击率', '点击转化率', '新用户占比']
    conversion_report[rate_columns2] = conversion_report[rate_columns2].apply(lambda x: (x * 100).round(2).astype(str) + '%')

    ws3 = wb.create_sheet(title='线上获客细则')
    for r in dataframe_to_rows(conversion_report, index=False, header=True):
        ws3.append(r)

    df2['日期'] = pd.to_datetime(df2['日期'])
    df2['时期'] = df2['日期'].apply(get_custom_week_period)
    df2['是否门店新用户'] = df2['是否新客']
    crowd_conversion_report = df2.groupby(['时期', '人群','是否新客','优惠券ID', '面额', '渠道','人群ID','城市']).agg({
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '交易订单量': 'sum',
        '交易新用户数': 'sum',
        '核销用户数': 'sum',
        '券补金额': 'sum'
    }).reset_index()

    ws4 = wb.create_sheet(title='人群漏斗')
    for r in dataframe_to_rows(crowd_conversion_report, index=False, header=True):
        ws4.append(r)

    channel_user = df2.groupby(['时期','城市', '是否新客', '渠道']).agg({
        '领券用户数': 'sum',
        '交易用户数': 'sum',
        '核销用户数': 'sum',
        '交易订单量': 'sum',
        '交易新用户数': 'sum'
    }).reset_index()

    ws5 = wb.create_sheet(title='渠道分新老')
    for r in dataframe_to_rows(channel_user, index=False, header=True):
        ws5.append(r)

    output_file_path = f"{output_folder}/周报.xlsx"
    wb.save(output_file_path)

    messagebox.showinfo("完成", f"处理完成！文件已保存至 {output_file_path}")

In [4]:
def select_file():
    file_path = filedialog.askopenfilename()
    input_entry.delete(0, tk.END)
    input_entry.insert(0, file_path)

In [5]:
def select_folder():
    folder_path = filedialog.askdirectory()
    output_entry.delete(0, tk.END)
    output_entry.insert(0, folder_path)

In [6]:
# 创建主窗口
root = tk.Tk()
root.title("无忧保洁周报工具")

# 输入文件选择
tk.Label(root, text="选择Excel文件:").grid(row=0, column=0, padx=10, pady=5)
input_entry = tk.Entry(root, width=40)
input_entry.grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="浏览", command=select_file).grid(row=0, column=2, padx=10, pady=5)

# 输出文件夹选择
tk.Label(root, text="选择保存位置:").grid(row=1, column=0, padx=10, pady=5)
output_entry = tk.Entry(root, width=40)
output_entry.grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="浏览", command=select_folder).grid(row=1, column=2, padx=10, pady=5)

# 运行按钮
tk.Button(root, text="运行", command=lambda: process_excel(input_entry.get(), output_entry.get())).grid(row=2, column=1, pady=20)

# 运行主循环
root.mainloop()

# 打包

安装pyinstaller<br>
pip install pyinstaller<br>
切换至当前所在的文件夹目录下<br>
pyinstaller --onefile --windowed 需要打包的文件名.py